# MIRMOS Thermal Load Calcs

For MSIP

In [1]:
import numpy as np
import scipy as sp
import scipy.integrate

import astropy

from astropy import units as ur
from astropy import constants as cc

from pylab import *
ion()

In [2]:


def eq(T, coeffs):
    lT = np.log10(T)
    
    log_k = 0
    for ix, c in enumerate(coeffs):
        log_k += c*lT**ix
    
    return 10**log_k
        

# FROM NIST
https://trc.nist.gov/cryogenics/materials/6061%20Aluminum/6061_T6Aluminum_rev.htm

## Al 6061
Thermal Conductivity
Specific Heat
UNITS	W/(m-K)	J/(kg-K)
a	0.07918	46.6467
b	1.0957	-314.292
c	-0.07277	866.662
d	0.08084	-1298.3
e	0.02803	1162.27
f	-0.09464	-637.795
g	0.04179	210.351
h	-0.00571	-38.3094
i	0	2.96344
data range
4-300	4-300
equation range
1-300	4-300
curve fit % error relative to data
0.5	5


## G-10
Thermal Conductivity
(Normal Direction)

Thermal Conductivity
(Warp Direction)
Specific Heat

UNITS	W/(m-K)	W/(m-K)	J/(kg•K)
a	-4.1236	-2.64827	-2.4083
b	13.788	8.80228	7.6006
c	-26.068	-24.8998	-8.2982
d	26.272	41.1625	7.3301
e	-14.663	-39.8754	-4.2386
f	4.4954	23.1778	1.4294
g	-0.6905	-7.95635	-0.24396
h	0.0397	1.48806	0.015236
i	0	-0.11701	0
data range
4-300	4-300	4-300
equation range
10-300	12-300	4-300
curve fit % error relative to data
5	5	2


In [3]:

# k is conductivty (W/(m K)), and h is speicfic heat (J/(kg K))
#
k_al6061 = [0.07918, 1.09570, -0.07277, 0.08084, 0.02803, -0.09464, 0.04179, -0.00571, 0]
h_al6061 = [46.6467, -314.292, 866.662, -1298.3, 1162.27, -637.795, 210.351, -38.3094, 2.96344]
k_g10 = [-4.1236, 13.788, -26.068, 26.272, -14.663, 4.4954, -0.6905, 0.0397]
h_g10 = [-2.4083, 7.6006, -8.2982, 7.3301, -4.2386, 1.4294, -0.24396, 0.015236]

In [4]:
eq(100, k_g10)

0.30959932133090096

In [5]:
def Heat(A, L, k, T1, T2):
    """ Return the conducitve load in Watts for element with area A (m^2), length L (m), conductivty coefficients K, from T1 to T2"""
    
    
    f = lambda x: eq(x, k)
    
    return A/L * sp.integrate.quad(f, T1, T2)[0] # In watt

In [6]:
np.round(sp.integrate.quad(lambda x: eq(x, k_al6061), 15, 300)[0])

32177.0

In [7]:
7.1e5/9.77e2

726.7144319344933

In [8]:
32177.0/111

289.8828828828829

In [9]:
eq(200, k_g10)

0.44471158743377776

In [10]:
0.01/1 * Heat(1, 1, k_g10, 120, 300)

0.83482613673497

# Conductive

In [12]:
A = np.pi*(25 *ur.mm)**2 * 10;
L = 0.2 * ur.meter;

a = A.to(ur.meter**2).value
l = L.to(ur.meter).value

cond = Heat(a, l, k_g10, 120, 300) * ur.Watt 

# Radiative



In [13]:
Ain = 20*ur.meter**2
Aout = 30*ur.meter**2

Tin = 120*ur.Kelvin
Tout = 300*ur.Kelvin

ein = 0.05
eout = 0.05

s = cc.sigma_sb

MLI_layers = 20


Awindow = (np.pi * (150*ur.mm)**2).to(ur.meter**2)
Aworld = Awindow # 10000 * ur.meter**2
N_window = 3

In [14]:
denom = 1/ein + Ain/Aout * (1/eout - 1)
Q = s*Ain*(Tout**4 - Tin**4) / denom / MLI_layers

In [15]:
rad_dewar = Q.to(ur.W)

In [21]:
denomwindow = 1/ein + Awindow/Aworld * (1/eout - 1)
rad_window = s*Ain*(Tout**4 - Tin**4) / denomwindow / N_window

In [22]:
rad_dewar+cond+rad_window

<Quantity 98.39898798 W>